In [41]:
import pandas as pd


In [42]:
execution_role = "arn:aws:iam::525102048888:role/wipcoe-pricing-real-time-moni-MonitorSageMakerRole-11WVCBI6XNZO1" # Step function IAM role ARN
baseconstloc='s3://wipcoe-ml-s3-config-bucket/monitoring/constraints.json'
basestatloc='s3://wipcoe-ml-s3-config-bucket/monitoring/statistics.json'
EndpointName='wipcoe-pricing-ml-training-pipeline-lr-endpoint'
monitorop='s3://wipcoe-ml-s3-config-bucket/monitoring/reports'
XGB_schedular_name='custom-pricing-real-time-monitor-pipeline-scheduler'
LR_schedular_name='custom-pricing-real-time-monitor-pipeline-scheduler'
v_region = 'us-east-1' # AWS region
sec_groups = ["sg-044e0e7ce4f5721c0"]
subnets = ["subnet-0cf0e3f46326aa259",
           "subnet-0156b7f5500cf0b78",
           "subnet-032420199163cff9b"]
config_bucket = "wipcoe-ml-s3-config-bucket"
schedule_expression="cron(0 * ? * * *)"

In [1]:
import boto3

client = boto3.client('sagemaker')
sec_groups = ["sg-044e0e7ce4f5721c0"]
subnets = ["subnet-0cf0e3f46326aa259",
           "subnet-0156b7f5500cf0b78",
           "subnet-032420199163cff9b"]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [8]:
client.list_monitoring_schedules()
#client.describe_monitoring_schedule(MonitoringScheduleName="MLOPsDemo-prod-DataQuality")

# client.delete_monitoring_schedule(MonitoringScheduleName= 'MLOPsDemo-staging-DataQuality')
# client.delete_monitoring_schedule(MonitoringScheduleName= 'MLOPsDemo-staging-ModelExplainability')


{'MonitoringScheduleSummaries': [{'MonitoringScheduleName': 'MLOPsDemo-prod-DataQuality',
   'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:525102048888:monitoring-schedule/mlopsdemo-prod-dataquality',
   'CreationTime': datetime.datetime(2022, 7, 19, 11, 14, 20, 681000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 7, 19, 17, 12, 50, 570000, tzinfo=tzlocal()),
   'MonitoringScheduleStatus': 'Scheduled',
   'EndpointName': 'MLOPsDemo-prod',
   'MonitoringJobDefinitionName': 'cfn-dataqualityjobdefini-taztxbwdledt',
   'MonitoringType': 'DataQuality'},
  {'MonitoringScheduleName': 'MLOPsDemo-prod-ModelExplainability',
   'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:525102048888:monitoring-schedule/mlopsdemo-prod-modelexplainability',
   'CreationTime': datetime.datetime(2022, 7, 19, 11, 14, 19, 542000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 7, 19, 17, 4, 54, 128000, tzinfo=tzlocal()),
   'MonitoringScheduleStatus': 'Scheduled

In [49]:
response = client.create_monitoring_schedule(
          MonitoringScheduleName=LR_schedular_name,#'wipcoe-pricing-real-time-monitor-pipeline-XGSchedular',
          MonitoringScheduleConfig={
              'ScheduleConfig': {
                  'ScheduleExpression': schedule_expression
              },
              'MonitoringJobDefinition': {
                  'BaselineConfig': {
                      'ConstraintsResource': {
                          'S3Uri': baseconstloc
                      },
                      'StatisticsResource': {
                          'S3Uri': basestatloc
                      }
                  },
                  'MonitoringInputs': [
                      {
                                  "EndpointInput": {
                                      "EndpointName": EndpointName,
                                      "LocalPath": "/opt/ml/processing/input/monitoring_dataset_input",
                                      "S3InputMode": "File",
                                      "S3DataDistributionType": "FullyReplicated",
                                  }
                              },
                  ],
                  'MonitoringOutputConfig': {
                      'MonitoringOutputs': [
                          {
                              "S3Output": {
                                  "S3Uri": monitorop,
                                  "LocalPath": "/opt/ml/processing/output",
                                  "S3UploadMode": "EndOfJob",
                              }
                          },
                      ],
                  },
                  'MonitoringResources': {
                      "ClusterConfig": {
                          "InstanceCount": 1,
                          "InstanceType": "ml.m5.xlarge",
                          "VolumeSizeInGB": 100,
                      }
                  },
                  'MonitoringAppSpecification': {
                      "ImageUri": "156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer",
                      "RecordPreprocessorSourceUri":"s3://wipcoe-ml-s3-config-bucket/monitoring/inferpreprocessingscript.py",
                  },
                  'StoppingCondition': {
                      'MaxRuntimeInSeconds': 3600
                  },
                  'NetworkConfig': {
                      'VpcConfig': {
                          "SecurityGroupIds": sec_groups,
                          "Subnets": subnets
                      }
                  },
                  'RoleArn': execution_role
              }
          }
        )
